In [1]:
import pandas as pd
import seaborn as sns
import pandasql as psql
import numpy as np
import matplotlib.pyplot as plt
from dython.nominal import identify_nominal_columns
from scipy import stats
from scipy.stats import chisquare
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
import mca
import prince

In [2]:
activity = pd.read_csv('original_data/activity.csv')
survey = pd.read_csv('original_data/survey.csv')

C:\Users\Eren\AppData\Local\Temp/ipykernel_16472/3715228674.py:2: DtypeWarning: Columns (314,391,392,789,791,792,1124,1125,1227,1228,1231,1232,1477,1478,1554,1555,1556,1557,1558,1559,1561,1568,1571,1678,1687,1745,1746,1818,1836,1837,1843,1844,1849,1851,1852,1853,2072,2116,2117,2264,2266,2280,2281,2282,2283,2289,2290,2483,2484,2501,2502,2508,2509,2699,2711,2712,2830) have mixed types. Specify dtype option on import or set low_memory=False.
  survey = pd.read_csv('original_data/survey.csv')


In [3]:
activity = activity.dropna()

In [4]:
activity_survey = pd.merge(activity, survey, on=['egoid'])
activity_survey.to_csv('merged_data/activity_survey.csv', index=False)

# activity_survey = pd.read_csv('merged_data/activity_survey.csv')

KeyboardInterrupt: 

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = []
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.append((cols[i], cols[j], df.iat[i, j]))
    return pairs_to_drop

def get_top_abs_correlations(df):
    abs_corr = df.corr().abs()
    # au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(abs_corr)
    return sorted(labels_to_drop, key=lambda tup: tup[2], reverse=True)
    # au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    # return au_corr[0:n]

In [ ]:
get_top_abs_correlations(activity_survey)

[('selsa_rom_5', 'club10rc_5', 1.0000000000000007),
 ('egoid', 'egoid', 1.0),
 ('complypercent', 'complypercent', 1.0),
 ('meanrate', 'meanrate', 1.0),
 ('sdrate', 'sdrate', 1.0),
 ('steps', 'steps', 1.0),
 ('floors', 'floors', 1.0),
 ('sedentaryminutes', 'sedentaryminutes', 1.0),
 ('lightlyactiveminutes', 'lightlyactiveminutes', 1.0),
 ('fairlyactiveminutes', 'fairlyactiveminutes', 1.0),
 ('veryactiveminutes', 'veryactiveminutes', 1.0),
 ('lowrangemins', 'lowrangemins', 1.0),
 ('fatburnmins', 'fatburnmins', 1.0),
 ('cardiomins', 'cardiomins', 1.0),
 ('peakmins', 'peakmins', 1.0),
 ('lowrangecal', 'lowrangecal', 1.0),
 ('fatburncal', 'fatburncal', 1.0),
 ('cardiocal', 'cardiocal', 1.0),
 ('peakcal', 'peakcal', 1.0),
 ('major1rc_1', 'major1rc_1', 1.0),
 ('major2rc_1', 'major2rc_1', 1.0),
 ('hsclub1rc_1', 'hsclub1rc_1', 1.0),
 ('hsclub2rc_1', 'hsclub2rc_1', 1.0),
 ('hsclub3rc_1', 'hsclub3rc_1', 1.0),
 ('hsclub4rc_1', 'hsclub4rc_1', 1.0),
 ('hsclub5rc_1', 'hsclub5rc_1', 1.0),
 ('MEQTotal_

In [ ]:
('peakcal', 'peakmins', 0.9607914723948829),
('cardiocal', 'cardiomins', 0.9567735185533744),
('lowrangemins', 'complypercent', 0.9412248931445473),
('fatburncal', 'fatburnmins', 0.8625015334223399),
('fatburncal', 'fairlyactiveminutes', 0.7715594542738567),
('lowrangecal', 'lowrangemins', 0.7548677042920645),
('club7rc_5', 'sedentarytime_scale_1', 0.7517370456401893),
('club7rc_5', 'SelfEff_diet_1_1', 0.7137024249691005),
('lowrangecal', 'complypercent', 0.6992630539562396),
('veryactiveminutes', 'steps', 0.6956130605019214),
('fatburnmins', 'meanrate', 0.6570696974480279),
('fatburncal', 'veryactiveminutes', 0.6524032785080631),
('lightlyactiveminutes', 'sedentaryminutes', 0.6356660009175586),
('veryactiveminutes', 'sdrate', 0.6299855083543254),
('sedentaryminutes', 'complypercent', 0.5962465713081001),
('veryactiveminutes', 'fairlyactiveminutes', 0.5895204991755101),
('fatburnmins', 'fairlyactiveminutes', 0.5893570134160409),
('club7rc_5', 'PSQI2_1', 0.5866469928455073),
('club7rc_5', 'SelfEff_diet_25_1', 0.5785041190899385),
('cardiomins', 'sdrate', 0.5452896691952817),
('steps', 'sdrate', 0.5447584588939157),
('fairlyactiveminutes', 'steps', 0.544264026067217),
('fatburncal', 'steps', 0.526351699286462),
('cardiocal', 'sdrate', 0.524690281916914),
('lowrangemins', 'sedentaryminutes', 0.5168834110893948),
('lowrangecal', 'sedentaryminutes', 0.5147434316635867),
('club7rc_5', 'physicalactivity5_1', 0.4970974096756166),
('lightlyactiveminutes', 'steps', 0.49213548001179924),
('club7rc_5', 'SelfEff_diet_26_1', 0.48919859480321704),
('fatburncal', 'meanrate', 0.48755164554996777),
('fatburncal', 'sdrate', 0.4833166996964153),
('lowrangecal', 'lightlyactiveminutes', 0.47880337436157105),
('club7rc_5', 'PSQI4_1', 0.466190532614564),
('cardiocal', 'veryactiveminutes', 0.4616353630381654),
('club7rc_5', 'PSQIGlobal_1', 0.46090807503070896),
('club7rc_5', 'SelfEff_diet_6_1', 0.4608923337198263),
('club7rc_5', 'selfreg_scale_1', 0.45844484476158903),
('floors', 'steps', 0.4581370720055981),
('club7rc_5', 'SelfEff_exercise_15_1', 0.4571867638156217),
('SelfEff_exercise_4_5', 'SelfEff_exercise_8_2', 0.45480189212367256),
('fairlyactiveminutes', 'sdrate', 0.45393586003552844),
('club7rc_5', 'HealthSatisfactionScale_1', 0.4370084816842345),
('club7rc_5', 'STAITraitTotal_1', 0.4369706217187404),
('club7rc_5', 'SelfEff_exercise_2_1', 0.4366539152846289),
('fatburnmins', 'steps', 0.4336378170574076),
('cardiomins', 'veryactiveminutes', 0.432529237283147),
('lightlyactiveminutes', 'complypercent', 0.42783671864101896),
('club7rc_5', 'SelfEff_exercise_9_1', 0.41635834982134223),
('club7rc_5', 'physicalactivity4_1', 0.4052891583291527),
('fatburncal', 'lightlyactiveminutes', 0.4038976459402531),
('lowrangemins', 'meanrate', 0.40130422373304436),
